In [1]:
pwd

'/home/jovyan/critical_feature_extraction/ipynb'

In [2]:
cd ..

/home/jovyan/critical_feature_extraction


In [28]:
%run __init__.py
from sklearn.decomposition import PCA, KernelPCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.metrics import r2_score, mean_squared_error

### Feature Extraction:
1. Check df and clean 
2. KPCA
3. Think of other non-linear dimension reduction tools.

In [4]:
apple = pd.read_csv('data/sandp500/individual_stocks_5yr_TECHNICALS_clean/AAPL_data.csv')

In [5]:
apple.head(1)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Date,Open,High,Low,Close,Volume,Name,...,RSI_12,STD_5,STD_20,SO%d_5,SO%d_20,SO%k,Trix_5,Trix_20,TSI_25_13,Target
0,61,61,61,2012-11-09,77.2,79.27,76.25,78.15,232476496,AAPL,...,0.071578,3.008766,4.566575,0.356641,0.372499,0.629139,-0.011429,-0.003303,-0.371455,0.779817


### Clean:
1. remove empty columns form df
2. make the date the index of the df

In [6]:
apple = apple.drop('Unnamed: 0', axis=1)
apple = apple.drop('Unnamed: 0.1', axis=1)
apple = apple.drop('Unnamed: 0.1.1', axis =1)

In [7]:
apple = apple.set_index(pd.to_datetime(apple.Date))
apple = apple.drop('Date', axis=1)
apple = apple.drop('Name', axis=1)

### KPCA:
1. Extract X and y values
2. train, test, split
3. Grid Search
    - build pipeline
    - build grid
    - preform gridsearch
    - fit pipe


In [8]:
y = apple['Target']
X = apple.drop('Target', axis=1)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)

In [15]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('kpca', KernelPCA()),
    ('svr', SVR())
])

In [20]:
kpca_params = {
    'kpca__n_components': np.arange(2, 11, 1),
    'kpca__kernel': ['rbf', 'sigmoid', 'cosine'],
    'kpca__gamma': np.linspace(.1, 1, 10),
    'svr__kernel':['sigmoid', 'rbf']
}

In [21]:
pipe_gs = GridSearchCV(pipeline, param_grid= kpca_params, cv=5, n_jobs=-1)

In [22]:
pipe_gs.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('kpca', KernelPCA(alpha=1.0, coef0=1, copy_X=True, degree=3, eigen_solver='auto',
     fit_inverse_transform=False, gamma=None, kernel='linear',
     kernel_params=None, max_iter=None, n_components=None, n_jobs=1,
     ran... epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False))]),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'kpca__n_components': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10]), 'kpca__kernel': ['rbf', 'sigmoid', 'cosine'], 'kpca__gamma': array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ]), 'svr__kernel': ['sigmoid', 'rbf']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [24]:
y_pred = pipe_gs.predict(X_test)

In [29]:
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_test, y_pred))

Mean squared error: 0.04
Variance score: 0.53
